# **Introduction to NEON AOP**

## Day 1 NEON AOP walkthrough

In this notebook, we'll walk through: <br>
1. Visualizing NEON AOP hyperspectral data
2. Masking data using QA bands
3. Exploring spectral signatures over space
4. Calculating vegetation indices
5. Zonal statistics using a landcover data product derived from satellite imagery (Landsat)


Acknowledgement:
> The National Ecological Observatory Network is a program sponsored by the National Science Foundation and operated under cooperative agreement by Battelle. This material is based in part upon work supported by the National Science Foundation through the NEON Program."

This notebook was adapted from [NEON Intro to AOP Data in Google Earth Engine](https://www.neonscience.org/resources/learning-hub/tutorials/intro-aop-data-google-earth-engine-tutorial-series). The [NEON learning hub](https://www.neonscience.org/resources/learning-hub) is a great resource to learn more!


In [1]:
import ee
import geemap
import geemap.colormaps as cm
ee.Authenticate()
ee.Initialize(project='511923968009')
geemap.ee_initialize(project='511923968009')

# Visualizing NEON AOP hyperspectral data

## Importing and viewing NEON AOP data

## Importing AOP data

In the code chunk below, we import 4 ImageCollections:<br>
  - [Surface Directional Reflectance](https://data.neonscience.org/data-products/DP3.30006.001)
  - [High resolution camera imagery, red-green-blue (RGB)](https://data.neonscience.org/data-products/DP3.30010.001)
  - [Canopy height model](https://data.neonscience.org/data-products/DP3.30015.001)
  - [Digital elevation model](https://data.neonscience.org/data-products/DP3.30024.001)

In [2]:
# Import the AOP surface directional reflectance (SDR)
aopSDR = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30006-001')
# Import the AOP red-green-blue imagery (RGB)
aopRGB = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30010-001')
# Import the AOP canopy height model data (CHM)
aopCHM = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30015-001')
# Import the AOP digital elevation model data (DEM)
aopDEM = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30024-001')

## Exploring the available AOP data

In the code chunk below, we print out the name of each SDR dataset currently available on GEE.

Then, we see how to filter the ImageCollection to only show Images that match our site of interest and/or dates of interest.

In [3]:
# See all SDR data currently uploaded
aopCHM.aggregate_array('system:index')

In [4]:
aopCHM.first()

In [5]:
# Filter the data to only dates or sites of interest

# read in a single SDR image at the NEON site JERC in 2018
HARV_CHM_2016 = aopCHM \
  .filterDate('2016-01-01', '2016-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'HARV') \
  .first()

HARV_CHM_2016

### Reflection questions (discuss in groups of 3)
1. Inspect the image that prints after the code chunk above, then answer the following:
  - How many bands does the 2018 JERC surface reflectance have? What does those bands represent?
  - What properties does the 2018 JERC surface reflectance have?
    - What do you think WL_FWHM_B### means? [(hint)](https://www.neonscience.org/resources/learning-hub/tutorials/hyper-spec-intro)


For question 2, divide up the 3 remaining data products (RGB, CHM, DEM) among group members.
2. Modify the code chunk above to repeat this process for 1 image from each of the 4 NEON AOP ImageCollections we imported. Take notes on the bands and properties associated with each ImageCollection. *HINT: You may need to look at sites beyond JERC. Check the data availability!*
  - What properties do you notice all Images have in common?
  - What is unique to specific data types?

## Visualizing

In [6]:
# Create variables for some of the images we may want to visualize
SOAP_2019_sdr = aopSDR \
  .filterDate('2019-01-01', '2019-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()

SOAP_2021_sdr = aopSDR \
  .filterDate('2021-01-01', '2021-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()

SOAP_2019_rgb = aopRGB \
  .filterDate('2019-01-01', '2019-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()

SOAP_2021_rgb = aopRGB \
  .filterDate('2021-01-01', '2021-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()

SOAP_2019_chm = aopCHM \
  .filterDate('2019-01-01', '2019-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()

SOAP_2021_chm = aopCHM \
  .filterDate('2021-01-01', '2021-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()

SOAP_2021_dem = aopDEM \
  .filterDate('2021-01-01', '2021-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()



In [7]:
SOAP_2019_rgb

In [8]:
# Set up visualization params
RGB_bands = ['B053', 'B035', 'B019'] # These are the band names for the red, green, blue bands in the SDR data
rgbVis = {'min': 0, 'max': 1260, 'gamma': 0.8} # This sets a nice range of values for mapping the RGB data
demVis = {'min': 0, 'max': 1260, 'gamma': 0.8} # Change these values to make your DEM layer look better. What do you think appropriate min/max values would be?

# Create maps
m = geemap.Map() # initialize the Map object

m.addLayer(SOAP_2021_dem, demVis, 'SOAP 2021 DEM') # Add the DEM
m.addLayer(SOAP_2021_sdr.select(RGB_bands), rgbVis, 'SOAP 2021 SDR RGB') # Add the red-green-blue bands from the SDR data
m.addLayer(SOAP_2019_rgb)
m.centerObject(SOAP_2021_dem, 12)


m # display the Map object

Map(center=[37.0473267491171, -119.25465121198724], controls=(WidgetControl(options=['position', 'transparent_…

## Reflection questions (think-pair-share)
1. Why does one of the layers looks pink when we first run this code?
2. What do you think would be appropriate min/max values for the DEM? Update your map to use these updated DEM values. *Hint: update the demVis variable.*
3. Why are there vertical stripes in the RGB imagery?
4. Try plotting a "false-color composite" using the SDR data. A popular false-color composite is NIR-R-G, which just means that we plot the near infrared (NIR), red, and green bands. Or, you can try SWIR-NIR-R *Hint: the SWIR band is 365, the NIR band is band 97*
  - [Here](https://earthobservatory.nasa.gov/features/FalseColor/page6.php) is helpful resource from NASA about popular false-color composites.
5. A significant event happened at Soaproot Saddle in 2020. Using the SOAP layers we've explored so far, try to see if you can figure out what happened!
  - What layers show the biggest changes in 2019 vs 2021?
  - How can you find more information to confirm your guess?

Don't worry if it's not yet clear what happened at SOAP in 2020 year. We'll come back to this question soon...

# Explore spectral signatures across a study site
In this section, we'll plot the spectral signature of different pixels. Don't worry if you don't quite follow every line of code below, but do explore the interactive map and test out different pixels!

This visualization was developed based on the following tutorials:<br>
[NEON Plot spectral signatures of AOP Reflectance data in GEE](https://www.neonscience.org/resources/learning-hub/tutorials/intro-aop-data-google-earth-engine-tutorial-series#toggle-28) <br>
[geemap GitHub page examples](https://github.com/gee-community/geemap/blob/master/examples/notebooks/geemap_and_earthengine.ipynb)

In [9]:
# Import SDR, RGB data
SOAP_2019_sdr = aopSDR \
  .filterDate('2019-01-01', '2019-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()

SOAP_2019_rgb = aopRGB \
  .filterDate('2019-01-01', '2019-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()

# Select the WL_FWHM_B*** band properties (using regex)
properties = SOAP_2019_sdr.toDictionary()
wl_fwhm_dict = properties.select(['WL_FWHM_B+\d{3}'])

# Pull out the wavelength, fwhm values to a list
wl_fwhm_list = wl_fwhm_dict.values()

# Function to pull out the wavelength values only and convert the string to float
def get_wavelengths(x):
  str_split = ee.String(x).split(',')
  first_elem = ee.Number.parse((str_split.get(0)))
  return first_elem

# apply the function to the wavelength full-width-half-max list
wavelengths = wl_fwhm_list.map(get_wavelengths)

In [16]:
from ipyleaflet import *
from bqplot import pyplot as plt

# Build map
m = geemap.Map()
m.default_style = {"cursor": "crosshair"}

# Set visualization parameters.
rgbVis = {'min': 0, 'max': 255, 'gamma': 0.8}
sdrVis = {'min': 103, 'max': 1160, 'gamma': 0.8}

# Add Earth Engine layers to Map
m.addLayer(SOAP_2019_sdr.select(RGB_bands), sdrVis, "SOAP 2019 SDR")
m.addLayer(SOAP_2019_rgb, rgbVis, "SOAP 2019 RGB")
m.centerObject(SOAP_2019_rgb)

figure = plt.figure(
    1,
    title="Reflectance",
    layout={"height": "200px", "width": "600px"},
)


# Set up variables for plotting markers, reflectance
x = wavelengths.getInfo()
coordinates = []
markers = []
marker_cluster = MarkerCluster(name="Marker Cluster")
m.add_layer(marker_cluster)

# Helper function called each time we click a new point
def handle_interaction(**kwargs):
    latlon = kwargs.get("coordinates")
    if kwargs.get("type") == "click":
      try:
        coordinates.append(latlon)
        markers.append(Marker(location=latlon))
        marker_cluster.markers = markers
        xy = ee.Geometry.Point(latlon[::-1])
        y = SOAP_2019_sdr.select('B.*').sample(xy, 1).first().toDictionary().values().getInfo()
        plt.clear()
        plt.plot(x, y)
      except:
        print('Please click a point inside the SOAP box.')


m.on_interaction(handle_interaction)

fig_control = WidgetControl(widget=figure, position="bottomright")
m.add_control(fig_control)

m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Reflection questions

Click around in the scene above. In your group, try to match each spectrum shown below with a category. Are there any other categories that interest you? Can you modify the code above to plot the JERC site?

CATEGORIES:
*   Water
*   Vegetation
*   Rocky outcropping

SPECTRA:<br>
1.
<img src='https://drive.google.com/uc?export=view&id=1FjKTtcKvsZa7OvHf2BcmgsR8JkJ7z5RM' width=400>

2.
<img src='https://drive.google.com/uc?export=view&id=1aMC-5xQjKdMfix2-CjahQZESebGSb5fc' width=400>

3.
<img src='https://drive.google.com/uc?export=view&id=1lPn0V0nezIR39xPbL78yfegZnVzzVqX2' width=400>

# Calculate vegetation indices

In the exercise above, you may have noticed some characteristic features of vegetation spectra, including chlorophyll and water absorption bands.<br>
<img src='https://drive.google.com/uc?export=view&id=1vCj_UtLDbiveY_Oyxw_sO7NUACug3DOX' width=400> Image credit: [NASA](https://science.nasa.gov/ems/01_intro)


Vegetation indices take advantage of some of these characteristics features of vegetation spectra. NDVI, the normalized difference vegetation index, is a common vegetation index. NDVI takes advantage of the characteristic "red edge"  between visible and near infrared bands.

<img src='https://drive.google.com/uc?export=view&id=1vdR7NPvynbN0OPrZYsdiCOVnTxTxo5sH' width=400> Image credit: [USDA, USFS Geospatial Technology and Applications Center Training Materials](https://fsapps.nwcg.gov/gtac/CourseDownloads/Training/Remote_Sensing/DigitalSoilMapping_EarthEngine_NRCS/03_Lectures_FINAL/FinalPDFs_lectures/Day1.3_RSIndices_FINAL.pdf)

Let's take a look at NDVI mapped over SOAP.

In [17]:
# Import JERC 2018 data
SOAP_2021 = ee.ImageCollection("projects/neon-prod-earthengine/assets/DP3-30006-001") \
  .filterDate('2021-01-01', '2021-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()

# Calculate NDVI
aop_ndvi = SOAP_2021.normalizedDifference(['B097', 'B055']).rename('aop_ndvi') # 'B097'=NIR; 'B055'=red


# Map NDVI for JERC

# set up visualization params
bwgPalette = [
  '#964B00',  # brown (low NDVI)
  'FFFFFF',   # white (med NDVI)
  '00ff00',   # green  (high NDVI)
]
ndviVis = {'min': 0.2, 'max': 1, 'palette': bwgPalette}

# create map
m = geemap.Map()

m.addLayer(SOAP_2021.select(RGB_bands), sdrVis, 'JERC 2018 RGB')
m.addLayer(aop_ndvi, ndviVis, 'JERC 2018 NDVI')

m.centerObject(SOAP_2021, 12)
m


Map(center=[37.04743330428505, -119.2490326540732], controls=(WidgetControl(options=['position', 'transparent_…

# Optional: Masking data

As we saw in the map above, there are some data quality issues in the raw imagery. Let's apply QA masks to our data and compare maps of the masked vs unmasked data.

In [12]:
SOAP_2019_sdr.select('[^B]*') # Select the non-reflectance bands (all the bands that don't start with B in this case)

## Reflection questions (work in groups of 3)
In the print-out of the image above, you can see 16 QA and metadata bands. For this example, we'll be using the Weather_Quality_Indicator.

1. Create a new code chunk. Using the code you saw in the sections above, select only the Weather_Quality_Indicator band. Then, create a map of that band. Are there any pixels impacted by poor weather in the 2019 SDR data? *Hint: You can use the color palette defined below*


```
# Define a palette for the weather - to match NEON AOP's weather color conventions
gyrPalette = [
  '00ff00', # green  (<10% cloud cover)
  'ffff00', # yellow (10-50% cloud cover)
  'ff0000'  # red    (>50% cloud cover)
]

weather_qaParams = {'min':1, 'max':3, 'palette': gyrPalette}
```



Now, we're ready to mask all pixels that have >10% cloud cover.  

Masking is the process of "hiding" pixels based on a mask layer, as shown in this image from NEON:
<img src='https://drive.google.com/uc?export=view&id=1YkSBHbodr6zLshOIFBac8AFRBJsMpSHT' width=600> Image credit: [NEON Mask Rasters tutorial](https://www.neonscience.org/resources/learning-hub/tutorials/mask-rasters-py)


In [13]:
# Create a raster where pixels=1 for <10% cloud cover and pixels=0 for >10% cloud cover. This is our mask.
soap_2019_clearWeather = SOAP_2019_sdr.select('Weather_Quality_Indicator').eq(1)

# Apply the clear weather mask to the original data
SOAP_2019_sdr_clear = SOAP_2019_sdr.updateMask(soap_2019_clearWeather)

# Map the unmasked and masked SDR data
m = geemap.Map() # initialize the Map object

m.addLayer(soap_2019_clearWeather, {'min':0, 'max':1}, 'mask')
m.addLayer(SOAP_2019_sdr.select(RGB_bands), rgbVis, 'unmasked SOAP 2019 SDR RGB') # Add the red-green-blue bands from the SDR data
m.addLayer(SOAP_2019_sdr_clear.select(RGB_bands), rgbVis, 'masked SOAP 2019 SDR RGB') # Add the red-green-blue bands from the SDR data
m.centerObject(SOAP_2019_sdr)

m # display the Map object

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# OPTIONAL: Another vegetation index
If we plot NDVI over JERC (as shown in the code chunk below), you may notice that NDVI gets pretty saturated over this very green site! Let's try another vegetation index, EVI (enhanced vegetation index), which tends to get less saturated.

In [14]:
# Import JERC 2018 data
JERC_2018 = ee.ImageCollection("projects/neon-prod-earthengine/assets/DP3-30006-001") \
  .filterDate('2018-01-01', '2018-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'JERC') \
  .first()

# Calculate NDVI
aop_ndvi = JERC_2018.normalizedDifference(['B097', 'B055']).rename('aop_ndvi') # 'B097'=NIR; 'B055'=red


# Map NDVI for JERC

# set up visualization params
bwgPalette = [
  '#964B00',  # brown (low NDVI)
  'FFFFFF',   # white (med NDVI)
  '00ff00',   # green  (high NDVI)
]
ndviVis = {'min': 0.2, 'max': 1, 'palette': bwgPalette}

# create map
m = geemap.Map()

m.addLayer(JERC_2018.select(RGB_bands), sdrVis, 'JERC 2018 RGB')
m.addLayer(aop_ndvi, ndviVis, 'JERC 2018 NDVI')

m.centerObject(JERC_2018, 12)
m


Map(center=[31.244748611840656, -84.46419349851583], controls=(WidgetControl(options=['position', 'transparent…

In [15]:
# Calculate EVI
JERC_2018_scaled = JERC_2018.divide(ee.Image.constant(JERC_2018.get('SCALING_FACTOR'))) # Use the scaled data (scaling doesn't cancel out in EVI calculation, as it did for NDVI)
aop_evi = JERC_2018_scaled.expression(
    '2.5 * ((NIR - Red) / (NIR + 6 * Red - 7.5 * Blue + 1))', {
      'NIR': JERC_2018_scaled.select('B097'),
      'Red': JERC_2018_scaled.select('B055'),
      'Blue': JERC_2018_scaled.select('B019')
    }).rename('EVI')


# Map EVI for JERC

# set up visualization params
bwgPalette = [
  '#964B00',  # brown (low EVI)
  'FFFFFF',   # white (med EVI)
  '00ff00',   # green  (high EVI)
]
eviVis = {'min': -0.1, 'max': 1, 'palette': bwgPalette} # EVI ranges from -1 to 1, but the min/max values for this image are 0.1-0.9, so adjusted for viewing

# create map
m = geemap.Map()

m.addLayer(JERC_2018.select(RGB_bands), sdrVis, 'JERC 2018 RGB')
m.addLayer(aop_evi, eviVis, 'JERC 2018 EVI')

m.centerObject(JERC_2018, 12)
m


Map(center=[31.244748611840656, -84.46419349851583], controls=(WidgetControl(options=['position', 'transparent…